<a href="https://colab.research.google.com/github/Jangchanhi/Skt_opencv/blob/main/Mediapipe_%EB%8F%99%EC%9E%91_%EB%B6%84%EB%A5%98_%EC%8B%A4%EC%8A%B5_%EC%98%88%EC%A0%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 라이브러리 설치하기

In [ ]:
!pip install opencv-python
!pip install numpy
!pip install mediapipe

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.5/33.5 MB 19.4 MB/s eta 0:00:00


In [ ]:
# import dependencies
from IPython.display import display, Javascript, Image
from google.colab.output import eval_js
from google.colab.patches import cv2_imshow
from base64 import b64decode, b64encode
import cv2
import numpy as np
import PIL
import io
import mediapipe as mp


# Video streaming functions

In [ ]:
# function to convert the JavaScript object into an OpenCV image
def js_to_image(js_reply):
  """
  Params:
          js_reply: JavaScript object containing image from webcam
  Returns:
          img: OpenCV BGR image
  """
  # decode base64 image
  image_bytes = b64decode(js_reply.split(',')[1])
  # convert bytes to numpy array
  jpg_as_np = np.frombuffer(image_bytes, dtype=np.uint8)
  # decode numpy array into OpenCV BGR image
  img = cv2.imdecode(jpg_as_np, flags=1)

  return img

# function to convert OpenCV Rectangle overlay image into base64 byte string
def convert_to_bytes(overlay_array):
  """
  Params:
          overlay_array: Numpy array (pixels) containing pose landmarks
  Returns:
        bytes: Base64 image byte string
  """
  # convert array into PIL image
  overlay_PIL = PIL.Image.fromarray(overlay_array, 'RGBA')
  iobuf = io.BytesIO()
  # format overlay into png for return
  overlay_PIL.save(iobuf, format='png')
  # format return string
  overlay_bytes = 'data:image/png;base64,{}'.format((str(b64encode(iobuf.getvalue()), 'utf-8')))

  return overlay_bytes

# JavaScript to properly create our live video stream using our webcam as input
def video_stream():
  js = Javascript('''
    var video;
    var div = null;
    var stream;
    var captureCanvas;
    var imgElement;
    var labelElement;

    var pendingResolve = null;
    var shutdown = false;

    function removeDom() {
       stream.getVideoTracks()[0].stop();
       video.remove();
       div.remove();
       video = null;
       div = null;
       stream = null;
       imgElement = null;
       captureCanvas = null;
       labelElement = null;
    }

    function onAnimationFrame() {
      if (!shutdown) {
        window.requestAnimationFrame(onAnimationFrame);
      }
      if (pendingResolve) {
        var result = "";
        if (!shutdown) {
          captureCanvas.getContext('2d').drawImage(video, 0, 0, 640, 480);
          result = captureCanvas.toDataURL('image/jpeg', 0.8)
        }
        var lp = pendingResolve;
        pendingResolve = null;
        lp(result);
      }
    }

    async function createDom() {
      if (div !== null) {
        return stream;
      }

      div = document.createElement('div');
      div.style.border = '2px solid black';
      div.style.padding = '3px';
      div.style.width = '100%';
      div.style.maxWidth = '600px';
      document.body.appendChild(div);

      const modelOut = document.createElement('div');
      modelOut.innerHTML = "<span>Status:</span>";
      labelElement = document.createElement('span');
      labelElement.innerText = 'No data';
      labelElement.style.fontWeight = 'bold';
      modelOut.appendChild(labelElement);
      div.appendChild(modelOut);

      video = document.createElement('video');
      video.style.display = 'block';
      video.width = div.clientWidth - 6;
      video.setAttribute('playsinline', '');
      video.onclick = () => { shutdown = true; };
      stream = await navigator.mediaDevices.getUserMedia(
          {video: { facingMode: "environment"}});
      div.appendChild(video);

      imgElement = document.createElement('img');
      imgElement.style.position = 'absolute';
      imgElement.style.zIndex = 1;
      imgElement.onclick = () => { shutdown = true; };
      div.appendChild(imgElement);

      const instruction = document.createElement('div');
      instruction.innerHTML =
          '<span style="color: red; font-weight: bold;">' +
          'When finished, click here or on the video to stop this demo</span>';
      div.appendChild(instruction);
      instruction.onclick = () => { shutdown = true; };

      video.srcObject = stream;
      await video.play();

      captureCanvas = document.createElement('canvas');
      captureCanvas.width = 640; //video.videoWidth;
      captureCanvas.height = 480; //video.videoHeight;
      window.requestAnimationFrame(onAnimationFrame);

      return stream;
    }
    async function stream_frame(label, imgData) {
      if (shutdown) {
        removeDom();
        shutdown = false;
        return '';
      }

      var preCreate = Date.now();
      stream = await createDom();

      var preShow = Date.now();
      if (label != "") {
        labelElement.innerHTML = label;
      }

      if (imgData != "") {
        var videoRect = video.getClientRects()[0];
        imgElement.style.top = videoRect.top + "px";
        imgElement.style.left = videoRect.left + "px";
        imgElement.style.width = videoRect.width + "px";
        imgElement.style.height = videoRect.height + "px";
        imgElement.src = imgData;
      }

      var preCapture = Date.now();
      var result = await new Promise(function(resolve, reject) {
        pendingResolve = resolve;
      });
      shutdown = false;

      return {'create': preShow - preCreate,
              'show': preCapture - preShow,
              'capture': Date.now() - preCapture,
              'img': result};
    }
    ''')

  display(js)

def video_frame(label, overlay):
  data = eval_js('stream_frame("{}", "{}")'.format(label, overlay))
  return data

# 웹캠영상 draw 예제 코드

In [29]:
# start streaming video from webcam
video_stream()
# label for video
label_html = 'Capturing...'

# Initialize MediaPipe Pose
overlay = ''

while True:
    js_reply = video_frame(label_html, overlay)
    if not js_reply:
        break

    # convert JS response to OpenCV Image
    image = js_to_image(js_reply["img"])

    # Run mediapipe

    # create transparent overlay for visualizing pose landmarks
    overlay_array = np.zeros([480,640,4], dtype=np.uint8)

    # Draw on overlay image

    overlay_array[:,:,3] = (overlay_array.max(axis = 2) > 0 ).astype(int) * 255
    # convert overlay of bbox into bytes
    overlay_bytes = convert_to_bytes(overlay_array)
    # update bbox so next frame gets new overlay
    overlay = overlay_bytes


<IPython.core.display.Javascript object>

In [33]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 새 섹션

In [30]:
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision

In [31]:
model_path = '/content/pose_landmarker_full.task'

In [22]:
BaseOptions = mp.tasks.BaseOptions
PoseLandmarker = mp.tasks.vision.PoseLandmarker
PoseLandmarkerOptions = mp.tasks.vision.PoseLandmarkerOptions
VisionRunningMode = mp.tasks.vision.RunningMode

options = PoseLandmarkerOptions(
    base_options=BaseOptions(model_asset_path=model_path),
    running_mode = VisionRunningMode.IMAGE)

detector = vision.PoseLandmarker.create_from_options(options)

In [25]:
# Load the input image.
numpy_image = cv2.imread("/content/male.jpg")
image_height, image_witdh = numpy_image.shape[:2]
mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=numpy_image)

In [26]:
detection_result = detector.detect(mp_image)
pose_landmarks_list = detection_result.pose_landmarks
landmarks = pose_landmarks_list[0]

In [27]:
# Print nose landmark
nose_landmark = landmarks[0]
print(nose_landmark)

NormalizedLandmark(x=0.5940964818000793, y=0.1386396288871765, z=-0.997409462928772, visibility=0.9999915361404419, presence=0.9999366998672485)


In [28]:
nose_landmark_x = int(nose_landmark.x * image_witdh)
nose_landmark_y = int(nose_landmark.y * image_height)
visibility = nose_landmark.visibility

print(f'Nose: ({nose_landmark_x}, {nose_landmark_y}, {visibility})')

Nose: (181, 84, 0.9999915361404419)
